### **Dynamic Capabilities**

In [0]:
from configs.paths import abfss_path, checkpoint_path

In [0]:
dbutils.widgets.text("file_name", "")
p_file_name = dbutils.widgets.get("file_name")

### **Data Reading**

In [0]:
df = spark.read.format("parquet").load(
    abfss_path("source", p_file_name)
)

In [0]:
df = (
    spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "parquet")
        .option(
            "cloudFiles.schemaLocation",
            checkpoint_path("bronze", p_file_name)
        )
        .load(
            abfss_path("source", p_file_name)
        )
)

### **Data Writing**

In [0]:
df.writeStream.format("parquet")\
    .option("checkpointLocation", checkpoint_path("bronze", p_file_name))\
    .option("path", abfss_path("bronze", p_file_name))\
    .trigger(once=True)\
    .start()

In [0]:
df = spark.read.format("parquet").load(abfss_path("bronze", p_file_name))